# Code your own polarizability tensor workflow, part 2

The main goal of this notebook is to show how the code implemented in the previous exercise can be easily converted to a fully functional class deriving from the `AbstractWorkflow` class. 

We will stick to the polarizability tensor problem, in order to build on what was achieved in the previous exercises. The structure of this notebook is very similar to the previous ones. You can use the same resources to help you (especially the), while the solution to this exercise can be found [here](https://mmoriniere.gitlab.io/MyBigDFT/notebooks/Solution_03.html).

## The [AbstractWorkflow](https://mmoriniere.gitlab.io/MyBigDFT/workflow.html#mybigdft.workflows.workflow.AbstractWorkflow) class

You already saw that using the `Workflow` class allows to get a clean separation of the procedure to compute a quantity of interest:
- first, the definition of the jobs to be run,
- then the jobs are run,
- and finally, the desired output can be computed from the results of the calculations. 

The goal of the `AbstractWorkflow` class is to ease the implementation of this type of workflow in a class that is meant to be general, *i.e.*, that can be applicable to a large class of systems, using a large set of possible input parameters. A class deriving from the `AbstractWorkflow` class is meant to be used simply by:
- initializing an instance of the class, setting the queue of jobs automatically,
- running all the jobs of the workflow instance thus created and performing the post-processing (performing all these actions is as simple as applying the `run` method).

And that's it! The post-processing procedure, which is defined in the class body, is automatically performed after all the jobs in the workflow queue. The quantities of interest (here, the polarizability tensor) are even used to set semi-private attributes, so that they can be easily accessed afterwards (don't worry, the last sentence should get clearer by the end of the notebook).

Another interest of writing a class is that it enables you to add more functionalities to your class afterwards, such as adding another quantity to compute while running the post-processing procedure. Such changes would be readily available in your scripts or notebooks where you used that class: no need to modify those files one by one to make that functionality available there.


To write such a class you will need to:
- define which are the quantities to be computed while post-processing the results by:
    * setting the value of the `POST_PROCESSING_ATTRIBUTES` list, containing the names of all the quantities of interest computed while post-processing,
    * defining a property getter for each of these attributes, so as to access them afterwards,
- override the `__init__` method, in order to make sure than the queue of jobs is correctly initialized,
- define the `post_proc` method to compute and then set the values of the quantities of interest.

## Create a class

The use of the `Workflow` class in the previous notebook showed you that having a clear separation of the initialization and post-processing of a workflow is important, notably to reduce the amount of code to copy and paste each time you want to change the system or the input parameters. We will go even further in that direction by writing a class deriving from the `AbstractWorkflow` class. You will see that the code you implemented in the previous exercise can be quickly adapted to meet our needs here.

Part of the body of the class is already written, you must only code the following:
- add the string `"pol_tensor"` to the POST_PROCESSING_ATTRIBUTES list,
- add an optional argument `ef_amplitude` to the `__init__` method to state the value of the electric field amplitude to be used in all the jobs (default value: `1.e-4`),
- initialize the queue of jobs in the `__init__` method,
- define the `post_proc` method to compute the value of the `pol_tensor` attribute.

The definition of the `pol_tensor` method is given so as to give you an example of how to actually code such methods. Note the use of a `_poltensor` attribute in that method and in the `post_proc` method; this is what we meant by "semi-private attributes" in the previous section: defining a property returning such a semi-private attribute ensures that you can access the `pol_tensor` attribute here, but it cannot be set. The only way of modifying the value of this attribute is to change the value of the `_poltensor`. This is actually what is done in the `post_proc` method!

## Use that class!

Having defined a class, you now are able to use it in order to effortlessly study some problems, such as those given below.

- ### The polarizability tensor must not depend on the rotation of the initial molecule

The atoms of the molecule here lie along the $z$ axis and the results should not change if the atoms lie along the $x$ and $y$ axis, for instance.

- ### What is the influence of the electric field amplitude on the results?

- ### How does the polarizability tensor depend on the system geometry?

You can modify the structure by changing the distance between the atoms.

## Improve that class!

One other interest of defining such a class is that you can easily improve it. You then have to do minor changes to your already performed scripts or notebooks to see those changes. For instance, you often find the mean polarizability in the literature instead of the tensor itself. This mean polarizability is defined as the mean value of the polarizability tensor diagonal elements. Given that you compute the polarizability tensor in the post-processing procedure, it is 

- ### Add the mean polarizability post-processing attribute

The goal here is to compute the mean polarizability while post-processing the calculations and make it available via an attribute. You therefore have to perform three changes to your class:
- add `"mean_polarizability"` to `POST_PROCESSING_ATTRIBUTES`
- define the `mean_polarizability` attribute as a property (hint: use `pol_tensor` as a template).
- compute the mean polarizabilty in the `post_proc` method (hint: the sum of the diagonal elements of a tensor can be easily computed with numpy: simply use ``np.trace()``) and set the `mean_polarizability` attribute via its private counterpart (hint: again, use what is done for `pol_tensor` as a template).

## Further step (optional)

You could add the possibility to perform two BigDFT calculations per space coordinates in order to get more accurate results, while removing the reference job without any electric field. This means that 6 jobs must be run instead of 4: two per space coordinate - one with a positive electric field amplitude, the other with a negative one (the run directory in that case must end by ``x-`` if the electric field is along the $x$ direction). This is especially relevant when the system under consideration has no dipole: BigDFT would still find a residual dipole, that might be large enough to give messy results (depending on the required accuracy). This can be tedious (it is but a variation of the present exercise), and you might want to use the same scheme as above: first use the Workflow class, and then implement that into the the ``PolTensor`` class. The difficulty is to keep the ability of running only four jobs to get the polarizability tensor and the mean polarizability: you will have to add an argument "order" to the ``__init__`` method and make sure it triggers to correct behaviour. You might therefore want to translate the workflow with 6 jobs to a new class deriving from AbstractWorkflow before actually modifying the ``PolTensor`` class. This should allow you to see which part of the code you have to modify, and to do it in a minimal fashion.